# Setup

In [1]:
from piper import piper
from piper.verbs import *

piper version 0.0.7, last run: Friday, 19 February 2021 11:34:19


# Test data 

In [2]:
%%piper
r812 <- pd.read_excel('inputs/R812.xlsx', sheet_name='query')
>> clean_columns() 
# >> count('sprint')
>> where("sprint == 'R812'")
# >> head()

# Setup

## Pivot table vs groupby

Pivot table vs groupby with multi-key and unstack and crosstab are functionally the same.<br>
Pivot allows automatic totals in both axes ... groupby does not.

### pivot_table

In [3]:
%%piper
r812.fillna('') >> pivot_table(index='functional_area',
                               columns='status',
                               values='object',
                               fill_value=0,
                               margins=True,
                               dropna=False,
                               aggfunc=np.size)

status,,Baxter Code Review Complete,Development Complete,L3 In Progress,L3 Pass,All
functional_area,,,,,,
ARO,0,2,0,8,4,14
Finance,0,5,1,0,0,6
Logistics,1,4,45,0,0,50
Order Management,0,0,10,0,0,10
eCommerce,0,0,8,0,0,8
All,1,11,64,8,4,88


### Groupby / unstack 

In [4]:
(r812.fillna('').groupby(['functional_area', 'status'])['status'].count()
 .unstack(level=1)
 .fillna(0)
 .astype(int))

status,,Baxter Code Review Complete,Development Complete,L3 In Progress,L3 Pass
functional_area,,,,,
ARO,0,2,0,8,4
Finance,0,5,1,0,0
Logistics,1,4,45,0,0
Order Management,0,0,10,0,0
eCommerce,0,0,8,0,0


### Crosstab

In [5]:
%%piper
pd.crosstab(r812.functional_area.fillna(''), r812.status.fillna(''))
>> adorn(axis='both')

,,Baxter Code Review Complete,Development Complete,L3 In Progress,L3 Pass,All
ARO,0,2,0,8,4,14
Finance,0,5,1,0,0,6
Logistics,1,4,45,0,0,50
Order Management,0,0,10,0,0,10
eCommerce,0,0,8,0,0,8
All,1,11,64,8,4,88


In [6]:
r812.query("~status.isin(['L3 In Progress', 'L3 Pass',\
           'Baxter Code Review Complete', 'Development Complete'])")

,object,gxp_critical_flag,object_type,fs_doc_nbr,ts_doc_nbr,sprint,testing_sprint,phase,description,status,...,retrofit_comments,sprint_track,test_script_id_92,test_script_name,version_projects,global,global_needed_in_production,number,item_type,path
2888,P564915A,NaN,APPL,NaN,NaN,R812,NaN,NaN,Work with Shipments - Baxter EMEA,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,Item,sites/erp92upgrade/erp92emea/Lists/EMEA Retrof...


## L2 Status

In [7]:
(r812.fillna('').groupby(['functional_area', 'l2_status'])['l2_status'].count()
 .unstack(level=0)
 .fillna(0)
 .astype(int))

functional_area,ARO,Finance,Logistics,Order Management,eCommerce
l2_status,,,,,
L2 Not required,14,6,50,10,8


In [8]:
%%piper
count(r812, 'l2_status', add_total=True)
# >> adorn(ignore_row_index=True)

,index,n
0,L2 Not required,88
1,Total,88
